# 0. Creating a clean dataframe
Due to the other Labs there are a lot of plots etc. we don't need no more.
So for this Lab we create a new clean dataframe.

In [1]:
import  numpy as np
import pandas as pd
import time
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
customer_df = pd.read_csv('./files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv')

In [3]:
# Snakecase
cols = []
for column in customer_df.columns:
    cols.append(column.lower())
customer_df.columns = cols
cols = []
for column in customer_df.columns:
    cols.append(column.replace(' ','_'))
customer_df.columns = cols

#### Numericals

In [4]:
# Change date columns to datetime
customer_df['effective_to_date'] = customer_df['effective_to_date'].apply(pd.to_datetime)

In [5]:
# Getting numerical data
customer_df_num = customer_df.select_dtypes(['number'])

In [6]:
# We assume that everything, that has more than 200 different values is continous data.
discrete_cols = []
conti_cols = []
# With this function we check if a column of our dataframe has more than 20 unique values and append
# the name of the column to the corresponding list
def iscontinous(column):
    if len(customer_df_num[column].unique()) > 200:
        conti_cols.append(column)
    else:
        discrete_cols.append(column)

In [7]:
# We apply the function to all the columns of the dataframe
for col in customer_df_num.columns:
    iscontinous(col)

In [8]:
# We create new dataframes for discrete and continous data with the lists
cont_data = customer_df_num[conti_cols]
disc_data = customer_df_num[discrete_cols]

#### Categoricals

In [9]:
# Getting Categorical data
categorical_df = customer_df.select_dtypes(include = object)

In [10]:
# Drop customer row, all unique values
categorical_df.drop('customer', inplace = True, axis = 1)

In [11]:
# Drop policy column because of redundancy
categorical_df.drop('policy_type', inplace = True, axis = 1)

In [12]:
# The States Wshington and Nevada have very few values. We aggregate them as 'others'.
categorical_df['state'] = categorical_df['state'].apply(lambda x: 'other' if x in ['Washington', 'Nevada'] else x)

In [13]:
# For education we can summarize Master and Doctor as higher education
categorical_df['education'] = categorical_df['education'].apply(lambda x: 'higher education' if x in ['Master', 'Doctor'] else x)

In [14]:
# For employment_status we can summarize Medical Leave, Retired and Disabled as other        
categorical_df['employmentstatus'] = categorical_df['employmentstatus'].apply(lambda x: 'other' if x in ['Medical Leave', 'Retired', 'Disabled'] else x)

In [15]:
# In the column policy, we still have 6 categories.Some have few values, we simplify Corporate and Secial
# For employment_status we can summarize Medical Leave, Retired and Disabled as other
def pol_change(x):
    if 'Corporate' in x:
        return 'Corporate'
    elif 'Special' in x:
        return 'Special'
    else:
        return x

categorical_df['policy'] = categorical_df['policy'].apply(pol_change)

In [16]:
# For vehicle_class we can summarize Sports Car, Luxury Car and Luxury SUV as Luxury Car    
categorical_df['vehicle_class'] = categorical_df['vehicle_class'].apply(lambda x: 'Luxury Car' if x in ['Luxury SUV', 'Sports Car'] else x)

# 1.In this final lab, we will model our data. 

In [17]:
pd.set_option('display.max_columns', None)
# First we must concatenate all our data again:
customer_numer = pd.concat([disc_data, cont_data], axis=1)
customer_clean = pd.concat([customer_numer, categorical_df], axis=1)
# Adding the effective_to_date columns
customer_clean['effective_to_date'] = customer_df['effective_to_date'] 
display(customer_clean.shape)
display(customer_clean.head(3))

(9134, 22)

,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,customer_lifetime_value,income,monthly_premium_auto,total_claim_amount,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size,effective_to_date
0,32,5,0,1,2763.519279,56274,69,384.811147,other,No,Basic,Bachelor,Employed,F,Suburban,Married,Corporate,Offer1,Agent,Two-Door Car,Medsize,2011-02-24
1,13,42,0,8,6979.535903,0,94,1131.464935,Arizona,No,Extended,Bachelor,Unemployed,F,Suburban,Single,Personal L3,Offer3,Agent,Four-Door Car,Medsize,2011-01-31
2,18,38,0,2,12887.431650,48767,108,566.472247,other,No,Premium,Bachelor,Employed,F,Suburban,Married,Personal L3,Offer1,Agent,Two-Door Car,Medsize,2011-02-19


Import sklearn train_test_split and separate the data.

In [18]:
# X-y split
y = customer_clean['total_claim_amount']
X = customer_clean.drop(['total_claim_amount'], axis=1)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. We will start with removing outliers, if you have not already done so. 
We have discussed different methods to remove outliers. 
Use the one you feel more comfortable with, define a function for that.
Use the function to remove the outliers and apply it to the dataframe.

In [20]:
# I decided to remove the outliers now, so that it doesn't affect the test data.
# There are some outliers for total_claim_amount, we don't touch them since that is our target.

# If we want to drop the outliers we have to temporarily reconcatenate X_train and y_train or
# the rows wouldn't match anymore.

In [21]:
X_temp = pd.concat([X_train,y_train], axis = 1 )
X_temp.shape

(7307, 22)

In [22]:
# We can see outliers above the 65000 mark for customer_lifetime_value.
len(X_temp[X_temp['customer_lifetime_value'] > 65000])

2

In [23]:
# They are just two rows, so we drop them.

In [24]:
X_temp = X_temp[X_temp['customer_lifetime_value'] < 65000]
X_temp.shape

(7305, 22)

In [25]:
# A lot of 0 values for income don't make sense and can also be seen as outliers.
# We use a function to fill them with the mean
mean = np.mean(X_temp['income'])
def fillmean(x):
    if x  == 0 :
        return mean
    return x
X_temp['income'] = X_temp['income'].apply(fillmean)
# Check if there are still 0's
display(X_temp[X_temp['income']==0])
X_temp.shape

,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,customer_lifetime_value,income,monthly_premium_auto,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size,effective_to_date,total_claim_amount


(7305, 22)

In [26]:
# Now we can split X and y again
y_train = X_temp['total_claim_amount']
X_train = X_temp.drop(['total_claim_amount'], axis=1)

# 3. Create a copy of the dataframe for the data wrangling.

In [27]:
y_trainw = y_train.copy()
y_testw = y_test.copy()
X_trainw = X_train.copy()
X_testw = X_test.copy()

In [28]:
X_testw

,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,customer_lifetime_value,income,monthly_premium_auto,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size,effective_to_date
708,1,49,0,1,4222.631209,74585,106,California,No,Basic,High School or Below,Employed,M,Rural,Married,Personal L3,Offer2,Agent,SUV,Medsize,2011-01-17
47,7,10,0,4,5514.344018,23218,71,Oregon,No,Basic,High School or Below,Employed,F,Suburban,Single,Personal L2,Offer4,Agent,Two-Door Car,Medsize,2011-01-02
3995,3,38,0,1,3808.122147,94903,94,Oregon,Yes,Extended,College,Employed,F,Suburban,Married,Personal L3,Offer2,Agent,Two-Door Car,Small,2011-01-28
1513,3,27,1,5,7914.823110,77153,99,California,No,Extended,College,Employed,F,Urban,Married,Personal L3,Offer1,Branch,Four-Door Car,Medsize,2011-02-09
3686,18,14,0,2,7931.722181,23769,69,other,No,Basic,High School or Below,other,F,Suburban,Divorced,Personal L2,Offer1,Branch,Four-Door Car,Medsize,2011-02-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4855,31,73,0,3,8688.754812,59098,110,Oregon,No,Basic,College,Employed,F,Suburban,Divorced,Personal L3,Offer2,Agent,SUV,Medsize,2011-01-27
1880,23,68,0,2,7623.580733,69900,64,Arizona,No,Basic,High School or Below,Employed,M,Rural,Married,Personal L2,Offer4,Branch,Two-Door Car,Large,2011-02-09
8472,24,11,0,1,4660.399573,89834,116,Oregon,No,Basic,Bachelor,Employed,F,Rural,Married,Corporate,Offer1,Branch,SUV,Medsize,2011-02-03
5967,30,6,1,1,2089.337182,0,63,other,No,Basic,High School or Below,Unemployed,M,Suburban,Single,Personal L2,Offer1,Agent,Four-Door Car,Small,2011-02-23


# 4. Normalize the continuous variables. 
You can use any one method you want.

In [29]:
# We have to again split everything into num_continuous and categoricals...
conti_cols = ['customer_lifetime_value', 'income', 'monthly_premium_auto']

In [30]:
X_testw[conti_cols]

,customer_lifetime_value,income,monthly_premium_auto
708,4222.631209,74585,106
47,5514.344018,23218,71
3995,3808.122147,94903,94
1513,7914.823110,77153,99
3686,7931.722181,23769,69
...,...,...,...
4855,8688.754812,59098,110
1880,7623.580733,69900,64
8472,4660.399573,89834,116
5967,2089.337182,0,63


In [31]:
X_trainw_numcont = X_trainw[conti_cols]
X_trainw_categ = X_trainw.drop(conti_cols, axis = 1)
X_testw_numcont = X_testw[conti_cols]
X_testw_categ = X_testw.drop(conti_cols, axis = 1)
display(X_trainw_numcont.head(3))
display(X_testw_numcont.head(3))

,customer_lifetime_value,income,monthly_premium_auto
5123,4768.955924,51144.00000,61
7738,2827.559570,37519.96345,79
214,2912.892035,81097.00000,74


,customer_lifetime_value,income,monthly_premium_auto
708,4222.631209,74585,106
47,5514.344018,23218,71
3995,3808.122147,94903,94


In [32]:
display(X_trainw_categ.head(3))
display(X_testw_categ.head(3))

,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size,effective_to_date
5123,28,32,1,9,other,No,Basic,Bachelor,Employed,M,Urban,Married,Personal L3,Offer1,Agent,Two-Door Car,Medsize,2011-01-10
7738,30,25,0,1,Oregon,No,Extended,College,Unemployed,F,Suburban,Single,Special,Offer3,Branch,Four-Door Car,Medsize,2011-02-18
214,7,67,0,1,Arizona,No,Basic,College,Employed,M,Suburban,Divorced,Personal L1,Offer4,Agent,Four-Door Car,Small,2011-02-28


,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size,effective_to_date
708,1,49,0,1,California,No,Basic,High School or Below,Employed,M,Rural,Married,Personal L3,Offer2,Agent,SUV,Medsize,2011-01-17
47,7,10,0,4,Oregon,No,Basic,High School or Below,Employed,F,Suburban,Single,Personal L2,Offer4,Agent,Two-Door Car,Medsize,2011-01-02
3995,3,38,0,1,Oregon,Yes,Extended,College,Employed,F,Suburban,Married,Personal L3,Offer2,Agent,Two-Door Car,Small,2011-01-28


In [33]:
# We use MinMax Scaler
from sklearn.preprocessing import MinMaxScaler

In [34]:
# Fitting the transformer
transformer = MinMaxScaler().fit(X_trainw_numcont)

In [35]:
# Scaling
X_train_scaled = pd.DataFrame(transformer.transform(X_trainw_numcont), columns=X_trainw_numcont.columns)
X_test_scaled = pd.DataFrame(transformer.transform(X_testw_numcont), columns=X_testw_numcont.columns)

In [36]:
X_train_scaled.head(3)

,customer_lifetime_value,income,monthly_premium_auto
0,0.045774,0.457029,0.000000
1,0.014820,0.305556,0.075949
2,0.016181,0.790047,0.054852


# 5. Encode the categorical variables 
(See the hint below for encoding categorical data!!!)

### 5.1 Ordinal encoding

In [37]:
# Ordinal encoding for
# coverage
# employmentstatus
# location code
# vehicle size

In [38]:
# Checking the unqiues
for i in ['coverage','employmentstatus', 'location_code', 'vehicle_size']:
    display(customer_clean[i].unique())

array(['Basic', 'Extended', 'Premium'], dtype=object)

array(['Employed', 'Unemployed', 'other'], dtype=object)

array(['Suburban', 'Rural', 'Urban'], dtype=object)

array(['Medsize', 'Small', 'Large'], dtype=object)

In [39]:
# We have to seperate out the columns we use for ordinal encoding
X_train_ordin = X_trainw_categ[['coverage','employmentstatus', 'location_code', 'vehicle_size']].copy()
X_test_ordin = X_testw_categ[['coverage','employmentstatus', 'location_code', 'vehicle_size']].copy()

In [40]:
X_train_ordin.head(2)

,coverage,employmentstatus,location_code,vehicle_size
5123,Basic,Employed,Urban,Medsize
7738,Extended,Unemployed,Suburban,Medsize


In [41]:
# Encoding
X_train_ordin["coverage"] = X_train_ordin["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})
X_test_ordin["coverage"] = X_test_ordin["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})

In [42]:
X_train_ordin["employmentstatus"] = X_train_ordin["employmentstatus"].map({"Unemployed" : 0, "other" : 1, "Employed" : 2})
X_test_ordin["employmentstatus"] = X_test_ordin["employmentstatus"].map({"Unemployed" : 0, "other" : 1, "Employed" : 2})

In [43]:
X_train_ordin["location_code"] = X_train_ordin["location_code"].map({"Rural" : 0, "Suburban" : 1, "Urban" : 2})
X_test_ordin["location_code"] = X_test_ordin["location_code"].map({"Rural" : 0, "Suburban" : 1, "Urban" : 2})

In [44]:
X_train_ordin["vehicle_size"] = X_train_ordin["vehicle_size"].map({"Small" : 0, "Medsize" : 1, "Large" : 2})
X_test_ordin["vehicle_size"] = X_test_ordin["vehicle_size"].map({"Small" : 0, "Medsize" : 1, "Large" : 2})

In [45]:
X_train_ordin.head(2)

,coverage,employmentstatus,location_code,vehicle_size
5123,0,2,2,1
7738,1,0,1,1


### 5.2 OneHot Encoding

In [46]:
X_train_hot = X_trainw_categ.drop(['coverage','employmentstatus', 'location_code', 'vehicle_size', 'effective_to_date'], axis = 1)
X_test_hot = X_testw_categ.drop(['coverage','employmentstatus', 'location_code', 'vehicle_size', 'effective_to_date'], axis = 1)

In [47]:
X_train_hot.head(2)

,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,state,response,education,gender,marital_status,policy,renew_offer_type,sales_channel,vehicle_class
5123,28,32,1,9,other,No,Bachelor,M,Married,Personal L3,Offer1,Agent,Two-Door Car
7738,30,25,0,1,Oregon,No,College,F,Single,Special,Offer3,Branch,Four-Door Car


In [48]:
# Onehot Encoding
from sklearn.preprocessing import OneHotEncoder

In [49]:
# Fitting the encoder
encoder = OneHotEncoder(drop='first').fit(X_train_hot)
column_name = encoder.get_feature_names_out(X_train_hot.columns)

In [50]:
# Encoding
X_train_hot = pd.DataFrame(encoder.transform(X_train_hot).toarray(), columns = column_name)
X_test_hot = pd.DataFrame(encoder.transform(X_test_hot).toarray(), columns = column_name)

# 6. The time variable can be useful. Try to transform its data into a useful one. 
Hint: Day week and month as integers might be useful.

In [51]:
# Converting for training data
X_train_date = pd.DataFrame([])
X_train_date['day'] = X_trainw_categ['effective_to_date'].dt.strftime('%d').copy()
X_train_date['month'] = X_trainw_categ['effective_to_date'].dt.strftime('%m').copy()
X_train_date['week'] = X_trainw_categ['effective_to_date'].dt.strftime('%w').copy()

In [52]:
# Converting for testing data
X_test_date = pd.DataFrame([])
X_test_date['day'] = X_testw_categ['effective_to_date'].dt.strftime('%d').copy()
X_test_date['month'] = X_testw_categ['effective_to_date'].dt.strftime('%m').copy()
X_test_date['week'] = X_testw_categ['effective_to_date'].dt.strftime('%w').copy()

In [53]:
# We have to convert everything to integers
def toint(x):
    x = str(x)
    if len(x) == 2 and x[0] == 0:
        return int(x[1])    
    return int(x)

for i in X_train_date.columns:
    X_train_date[i] = X_train_date[i].apply(toint)

for i in X_test_date.columns:
    X_test_date[i] = X_test_date[i].apply(toint)

In [54]:
X_train_date.dtypes

day      int64
month    int64
week     int64
dtype: object

In [55]:
X_train_date

,day,month,week
5123,10,1,1
7738,18,2,5
214,28,2,1
8580,8,1,6
7857,18,1,2
...,...,...,...
5734,29,1,6
5191,5,2,6
5390,27,1,4
860,27,2,0


# 7. Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

In [56]:
# We reconcatenate everythig:
# The OneHot encoded, the dates, the ordinal encoded and the transformed

In [57]:
# We have to reset the indexes, so that the encoded and transformed data matches with the others
# after concatenating
X_train_date.reset_index(inplace = True)
X_train_hot.reset_index(inplace = True)
X_train_ordin.reset_index(inplace = True)
X_train_scaled.reset_index(inplace = True)

X_test_date.reset_index(inplace = True)
X_test_hot.reset_index(inplace = True)
X_test_ordin.reset_index(inplace = True)
X_test_scaled.reset_index(inplace = True)

In [58]:
# Concatenating
X_train = pd.concat([X_train_date, X_train_hot, X_train_ordin, X_train_scaled], axis=1)
X_test = pd.concat([X_test_date, X_test_hot, X_test_ordin, X_test_scaled], axis=1)

In [59]:
X_train.head(2)

,index,day,month,week,index,months_since_last_claim_1,months_since_last_claim_2,months_since_last_claim_3,months_since_last_claim_4,months_since_last_claim_5,months_since_last_claim_6,months_since_last_claim_7,months_since_last_claim_8,months_since_last_claim_9,months_since_last_claim_10,months_since_last_claim_11,months_since_last_claim_12,months_since_last_claim_13,months_since_last_claim_14,months_since_last_claim_15,months_since_last_claim_16,months_since_last_claim_17,months_since_last_claim_18,months_since_last_claim_19,months_since_last_claim_20,months_since_last_claim_21,months_since_last_claim_22,months_since_last_claim_23,months_since_last_claim_24,months_since_last_claim_25,months_since_last_claim_26,months_since_last_claim_27,months_since_last_claim_28,months_since_last_claim_29,months_since_last_claim_30,months_since_last_claim_31,months_since_last_claim_32,months_since_last_claim_33,months_since_last_claim_34,months_since_last_claim_35,months_since_policy_inception_1,months_since_policy_inception_2,months_since_policy_inception_3,months_since_policy_inception_4,months_since_policy_inception_5,months_since_policy_inception_6,months_since_policy_inception_7,months_since_policy_inception_8,months_since_policy_inception_9,months_since_policy_inception_10,months_since_policy_inception_11,months_since_policy_inception_12,months_since_policy_inception_13,months_since_policy_inception_14,months_since_policy_inception_15,months_since_policy_inception_16,months_since_policy_inception_17,months_since_policy_inception_18,months_since_policy_inception_19,months_since_policy_inception_20,months_since_policy_inception_21,months_since_policy_inception_22,months_since_policy_inception_23,months_since_policy_inception_24,months_since_policy_inception_25,months_since_policy_inception_26,months_since_policy_inception_27,months_since_policy_inception_28,months_since_policy_inception_29,months_since_policy_inception_30,months_since_policy_inception_31,months_since_policy_inception_32,months_since_policy_inception_33,months_since_policy_inception_34,months_since_policy_inception_35,months_since_policy_inception_36,months_since_policy_inception_37,months_since_policy_inception_38,months_since_policy_inception_39,months_since_policy_inception_40,months_since_policy_inception_41,months_since_policy_inception_42,months_since_policy_inception_43,months_since_policy_inception_44,months_since_policy_inception_45,months_since_policy_inception_46,months_since_policy_inception_47,months_since_policy_inception_48,months_since_policy_inception_49,months_since_policy_inception_50,months_since_policy_inception_51,months_since_policy_inception_52,months_since_policy_inception_53,months_since_policy_inception_54,months_since_policy_inception_55,months_since_policy_inception_56,months_since_policy_inception_57,months_since_policy_inception_58,months_since_policy_inception_59,months_since_policy_inception_60,months_since_policy_inception_61,months_since_policy_inception_62,months_since_policy_inception_63,months_since_policy_inception_64,months_since_policy_inception_65,months_since_policy_inception_66,months_since_policy_inception_67,months_since_policy_inception_68,months_since_policy_inception_69,months_since_policy_inception_70,months_since_policy_inception_71,months_since_policy_inception_72,months_since_policy_inception_73,months_since_policy_inception_74,months_since_policy_inception_75,months_since_policy_inception_76,months_since_policy_inception_77,months_since_policy_inception_78,months_since_policy_inception_79,months_since_policy_inception_80,months_since_policy_inception_81,months_since_policy_inception_82,months_since_policy_inception_83,months_since_policy_inception_84,months_since_policy_inception_85,months_since_policy_inception_86,months_since_policy_inception_87,months_since_policy_inception_88,months_since_policy_inception_89,months_since_policy_inception_90,months_since_policy_inception_91,months_since_policy_inception_92,months_si

In [60]:
# Now we check the dtypes
X_train.dtypes
# Looks good

index                        int64
day                          int64
month                        int64
week                         int64
index                        int64
                            ...   
vehicle_size                 int64
index                        int64
customer_lifetime_value    float64
income                     float64
monthly_premium_auto       float64
Length: 184, dtype: object

# 8. Try a simple linear regression with all the data to see whether we are getting good results.

In [61]:
from sklearn import linear_model
lm = linear_model.LinearRegression()

In [63]:
# Fitting the model
lm.fit(X_train,y_train)
predictions = lm.predict(X_test)

In [64]:
# Assesing the model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, mean_absolute_error
# Training Data
# R2-Score
predictions = lm.predict(X_train)
r2score = r2_score(y_train, predictions)
# Mean Square Error
mse=mean_squared_error(y_train,predictions)
# Root Mean Square Error
rmse = np.sqrt(mse)
# Mean Absolute Error
mae = mean_absolute_error(y_train, predictions)

In [65]:
# Test Data
# R2-Score
predictions = lm.predict(X_test)
r2scoret = r2_score(y_test, predictions)
# Mean Square Error
mset=mean_squared_error(y_test,predictions)
# Root Mean Square Error
rmset = np.sqrt(mset)
# Mean Absolute Error
maet = mean_absolute_error(y_test, predictions)

In [67]:
display(pd.DataFrame({'ValType': ['Train', 'Test'], 'R2-Score': [r2score, r2scoret], 'MSE': [mse, mset] , 'RMSE': [rmse, rmset], 'MAE': [mae, maet]}))

,ValType,R2-Score,MSE,RMSE,MAE
0,Train,0.640151,30687.890589,175.179595,127.562081
1,Test,0.609334,31062.240379,176.244831,132.442633


# 9. Great! Now define a function that takes a list of models and train (and tests) them so we can try a lot of them without repeating code.

In [79]:
# Making the function
def modeler(models):
    results = []
    for model in models:
        # Fitting
        model.fit(X_train,y_train)
        # Evaluating
        # R2-Score
        predictions = model.predict(X_train)
        r2score = r2_score(y_train, predictions)
        # Mean Square Error
        mse=mean_squared_error(y_train,predictions)
        # Root Mean Square Error
        rmse = np.sqrt(mse)
        # Mean Absolute Error
        mae = mean_absolute_error(y_train, predictions)
        # Test Data
        # R2-Score
        predictions = model.predict(X_test)
        r2scoret = r2_score(y_test, predictions)
        # Mean Square Error
        mset=mean_squared_error(y_test,predictions)
        # Root Mean Square Error
        rmset = np.sqrt(mset)
        # Mean Absolute Error
        maet = mean_absolute_error(y_test, predictions)
        val = pd.DataFrame({'ValType': ['Train', 'Test'], 'R2-Score': [r2score, r2scoret], 'MSE': [mse, mset] , 'RMSE': [rmse, rmset], 'MAE': [mae, maet]})
        print(model)
        display(val)
        results.append(val)
    return results   

# 10. Use the function to check LinearRegressor and KNeighborsRegressor.

In [90]:
modellist = []

In [91]:
# Defining the models
line_reg = linear_model.LinearRegression()
modellist.append(line_reg)

In [92]:
from sklearn.neighbors import KNeighborsRegressor
kn_reg = KNeighborsRegressor(n_neighbors=3)
modellist.append(kn_reg)

In [93]:
# Passing them into the function
results = modeler(modellist)

LinearRegression()


,ValType,R2-Score,MSE,RMSE,MAE
0,Train,0.640151,30687.890589,175.179595,127.562081
1,Test,0.609334,31062.240379,176.244831,132.442633


KNeighborsRegressor(n_neighbors=3)


,ValType,R2-Score,MSE,RMSE,MAE
0,Train,0.346842,55701.348061,236.011330,175.042090
1,Test,-0.316858,104704.778553,323.581178,240.242746


# 11. You can check also the MLPRegressor for this task!

In [95]:
from sklearn.neural_network import MLPRegressor
mlp_reg = MLPRegressor(random_state=1, max_iter=500)
modellist.append(mlp_reg)

In [97]:
results = modeler(modellist)

LinearRegression()


,ValType,R2-Score,MSE,RMSE,MAE
0,Train,0.640151,30687.890589,175.179595,127.562081
1,Test,0.609334,31062.240379,176.244831,132.442633


KNeighborsRegressor(n_neighbors=3)


,ValType,R2-Score,MSE,RMSE,MAE
0,Train,0.346842,55701.348061,236.011330,175.042090
1,Test,-0.316858,104704.778553,323.581178,240.242746


MLPRegressor(max_iter=500, random_state=1)


,ValType,R2-Score,MSE,RMSE,MAE
0,Train,0.536577,39520.761762,198.798294,145.354651
1,Test,0.530655,37318.095823,193.178922,140.053451


# 12. Check and discuss the results.

It seems, that for this dataset with the specified transformations we achieve the best results with
the linear regression model.
The KN_Regressor apparently has problems with the data, this is because it works worse with a lot of
Dimensions, wich we introduced with encoding the discrete numerical data.
In a real life scenario further select features and maybe try another threshhold or binning for the 
discrete numerical data.